In [4]:
# !pip install nest_asyncio
# !pip install newspaper3k feedparser psycopg2
import nest_asyncio
nest_asyncio.apply()

In [67]:
x = [datetime.datetime, datetime.datetime.now()]
x.sort()
x

[datetime.datetime(2020, 4, 5, 14, 54, 45, 702024),
 datetime.datetime(2020, 4, 5, 14, 54, 45, 702049)]

In [63]:
from dataclasses import dataclass, field, astuple, asdict
import datetime

@dataclass
class Rss:
#     id: int = None
    url: str
    ticker: str = None
    fetched_at: datetime.datetime = None
#     fetched_at: datetime.datetime = field(default_factory=datetime.datetime.now, init=False)
    id: int = None

# e1 = Rss(url="aaa.com")
# e2 = Rss(url="aaa.com")
# e.ticker
# e.fetched_at
# print(e1, e2)
# for f in fields(e1):
#     print(f.name)
# astuple(e1)
a = ("aaa.com", None, None, 1)
e = Rss(*a)
e
# astuple(e)[:-1]
# asdict(e)
# a[:-1]
# Rss()
# a[1:]


Rss(url='aaa.com', ticker=None, fetched_at=None, id=1)

In [9]:
import pandas as pd
df = pd.read_csv('/workspace/twint/rss/resource/rss.csv')
for index, r in df.iterrows():
    print(r['ticker'], r['url'])

yhoo http://finance.yahoo.com/rss/headline?s=yhoo
tivo http://finance.yahoo.com/rss/headline?s=tivo


In [38]:
import datetime
import psycopg2
from psycopg2 import sql
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

CREATE_TABLE_RSS = """
    CREATE TABLE IF NOT EXISTS
        Rss(
            id serial PRIMARY KEY,
            url VARCHAR (255) UNIQUE NOT NULL,
            ticker VARCHAR (20),
            fetched_at TIMESTAMP
        );
"""

CREATE_TABLE_FEEDS = """
    CREATE TABLE IF NOT EXISTS
        feeds(
            id serial PRIMARY KEY,
            rss_id INTEGER REFERENCES Rss(id),
            url VARCHAR (255) NOT NULL,
            title VARCHAR (255),
            body TEXT,
            created_at TIMESTAMP NOT NULL DEFAULT NOW()
        );
"""

try:
    conn = psycopg2.connect(dbname="test", user="sqlpad", password="sqlpad", host="pg")
except: 
    conn = psycopg2.connect(user="sqlpad", password="sqlpad", host="pg")
    conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

    cur = conn.cursor()
    # cur.execute('CREATE DATABASE {};'.format(self.db_name))
    cur.execute(sql.SQL("CREATE DATABASE {}").format(sql.Identifier("test")))

with conn:
    with conn.cursor() as cur:
#         cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")
#         cur.execute("INSERT INTO test (num, data) VALUES (%s, %s)", (100, "abc'def"))
#         cur.execute(CREATE_TABLE_RSS)
#         cur.execute(CREATE_TABLE_FEEDS)
#         cur.execute("""
#             INSERT INTO rss (url, ticker, fetched_at)
#             VALUES (%s, %s, %s);""", 
#             ("http://aaa.bbb", "YHOO", datetime.datetime.now()))

        cur.execute("SELECT * FROM rss WHERE ticker=%s", ("YHOO", ))
        res = cur.fetchall()
        print(res)
#         print(_id)

#         cur.execute("""
#             INSERT INTO feeds (rss_id, url, title, body)
#             VALUES (%s, %s, %s, %s);""", 
#             (2, "http://aaa.bbb/123/", "a title", "a body"))
        
conn.close()
# cur.execute("CREATE TABLE test (id serial PRIMARY KEY, num integer, data varchar);")
# conn = psycopg2.connect(user="sqlpad", password="sqlpad", host="pg")


[(1, 'http://aaa.bbb', 'YHOO', datetime.datetime(2020, 4, 4, 15, 54, 22, 750328))]


In [ ]:
CREATE_TABLE_RSS = """
    CREATE TABLE IF NOT EXISTS
        Rss(
            id serial PRIMARY KEY,
            url VARCHAR (255) UNIQUE NOT NULL,
            ticker VARCHAR (20),
            fetched_at TIMESTAMP
        );
"""

CREATE_TABLE_FEEDS = """
    CREATE TABLE IF NOT EXISTS
        Feeds(
            id serial PRIMARY KEY,
            rss_id INTEGER REFERENCES Rss(id),
            url VARCHAR (255) NOT NULL,
            title VARCHAR (255),
            body TEXT,
            created_at TIMESTAMP NOT NULL DEFAULT NOW()
        );
"""

In [40]:
import asyncio
import functools

async def fetch(session, url: str):
    await asyncio.sleep(1)
    return "html<{}>".format(url)

def parse(html: str):
    return "parsed <{}>".format(html)

def save(doc):
    print("saved: {}".format(doc))


def query_rss(url):
    with conn:
        with conn.cursor() as cur:
            cur.execute("SELECT * FROM rss WHERE url=%s", (url,))
            res = cur.fetchall()
    return res

# def inert_rss():
#     with conn:
#         with conn.cursor() as cur:
#             cur.execute("SELECT * FROM rss WHERE url=%s", (url,))
#             res = cur.fetchall()

async def scrape_articles(session, urls):
    async def scrape(url):
        # is fetched?
        if es.has(clean(url))
            return
        
        async with session.get(url) as resp:
            print(resp.status)
            html = await resp.text()
            article.set_html(html)
            article.parse()
            es.index(article)
        
    asyncio.gather(*[scrape(url) for url in urls])    
        
async def scrape(url, session, conn):
    # fetch now or until when
    res = query_rss(url)
    try:
        d = res[0]
#         secs = d["fetched_at"] + d["freq"] - now
        secs= d[3]-d[2]-now
        await asyncio.sleep(secs)
    except IndexError:
        # create new rss entity
        with conn:
            with conn.cursor() as cur:
                cur.execute(
                    "INSERT INTO rss (url, ticker, fetched_at) VALUES (%s, %s, %s) RETURNING id;", 
                    (url, "YHOO", datetime.datetime.now())
                )
    async with session.get(url) as resp:
        print(resp.status)
        html = await resp.text()
#         rss = parse_rss(html)
        
        with conn:
            with conn.cursor() as cur:
                cur.execute("UPDATE rss SET fetched_at=now() WHERE ID=%s;", (d[0]))
                cur.execute("SELECT * FROM feeds ORDER BY published_at DESC LIMIT 1")
                res = cur.fetchall()
                
                for e in rss['entries']:
                    pass
#                     cur.execute("INSERT INTO feeds (url, title, content, created_at) VALUES (%s, %s, %s) RETURNING id;",
#                                 (d[0]))
                
        
#         rss = parse_rss(html)
#         save_rss(rss)
#         await asyncio.gather(*[scrape_article(e["url"]) for e in rss.entries])
    
async def main():
    urls = ["http://finance.yahoo.com/rss/headline?s=yhoo",
            "http://finance.yahoo.com/rss/headline?s=dis"]
    await asyncio.gather(*[scrape(url) for url in urls])
    
asyncio.run(main())

SyntaxError: invalid syntax (<ipython-input-40-b47f71fd4099>, line 31)

In [42]:
import asyncio
import functools

async def fetch(session, url: str):
    await asyncio.sleep(1)
    return "html<{}>".format(url)

def parse(html: str):
    return "parsed <{}>".format(html)

def query():
    with conn:
        with conn.cursor() as cur:
            cur.execute("SELECT id FROM rss WHERE ticker = 'YHOO'")
            _id,  = cur.fetchone()

def save(doc):
    print("saved: {}".format(doc))

async def scrape_articles(session, urls):
    async def scrape(url):
        # is fetched?
        if es.has(clean(url))
            return
        
        async with session.get(url) as resp:
            print(resp.status)
            html = await resp.text()
            article.set_html(html)
            article.parse()
            es.index(article)
        
    asyncio.gather(*[scrape(url) for url in urls])    
        
async def scrape(url, session):
    # fetch now or until when
    d = es.get({"url": url})
    if d is not None:
        secs = d["fetched_at"] + d["freq"] - now
        await asyncio.sleep(secs)
    else:
        es.index({"url": url})

#     html = await fetch(url)
    async with session.get(url) as resp:
        print(resp.status)
        html = await resp.text()
        rss = parse_rss(html)
        save_rss(rss)
        await asyncio.gather(*[scrape_article(e["url"]) for e in rss.entries])
    
async def main():
    urls = ["abc.com", "opq.com"]
    
    async def scrape(url):
                d = es.get({"url": url})
         = d["last_fetched"]

        html = await fetch(url)
        rss = parse_rss(html)
        save_rss(rss)
        await asyncio.gather(*[scrape_article(e["url"]) for e in rss.entries])
    
    await asyncio.gather(*[scrape(url) for url in urls])
    
#     task = asyncio.create_task(fetch())
#     task.add_done_callback(parse)
#     await task

asyncio.run(main())

saved: parsed <html<abc.com>>
saved: parsed <html<opq.com>>


In [16]:
import aiohttp
import feedparser

headers = {
#     "If-None-Match": "246ce00049b1df9930d4dc344180574f"
}

async def get():
    async with aiohttp.ClientSession() as session:
#         async with session.get('http://finance.yahoo.com/rss/industry?s=yhoo', headers=headers) as resp:
        async with session.get('http://finance.yahoo.com/rss/headline?s=dis', headers=headers) as resp:
            print(resp.status)
            print(resp.headers)
            t = await resp.text()
            return feedparser.parse(t)

await get()

200
<CIMultiDictProxy('X-Content-Type-Options': 'nosniff', 'X-Yahoo-Request-Id': 'fuuk2p1f8g7n2', 'Cache-Control': 'max-age=300, stale-while-revalidate=75', 'Content-Encoding': 'gzip', 'Content-Type': 'text/xml;charset=utf-8', 'Vary': 'Origin', 'Content-Length': '5707', 'Date': 'Sat, 04 Apr 2020 05:45:06 GMT', 'Age': '0', 'Strict-Transport-Security': 'max-age=31536000', 'Server': 'ATS', 'Expect-CT': 'max-age=31536000, report-uri="http://csp.yahoo.com/beacon/csp?src=yahoocom-expect-ct-report-only"', 'Public-Key-Pins-Report-Only': 'max-age=2592000; pin-sha256="2fRAUXyxl4A1/XHrKNBmc8bTkzA7y4FB/GLJuNAzCqY="; pin-sha256="I/Lt/z7ekCWanjD0Cvj5EqXls2lOaThEA0H2Bg4BT/o="; pin-sha256="K87oWBWM9UZfyddvDfoxL+8lpNyoUB2ptGtn0fv6G2Q="; pin-sha256="Wd8xe/qfTwq3ylFNd3IpaqLHZbh2ZNCLluVzmeNkcpw="; pin-sha256="WoiWRyIOVNa9ihaBciRSC7XHjliYS9VwUGOIud4PB18="; pin-sha256="cGuxAXyFXFkWm61cF4HPWX8S0srS9j0aSqN0k4AP+4A="; pin-sha256="dolnbtzEBnELx/9lOEQ22e6OZO/QNb6VSSX2XHA3E7A="; pin-sha256="i7WTqTvh0OioIruIfFR4kM

{'feed': {'rights': 'Copyright (c) 2020 Yahoo! Inc. All rights reserved.',
  'rights_detail': {'type': 'text/plain',
   'language': None,
   'base': '',
   'value': 'Copyright (c) 2020 Yahoo! Inc. All rights reserved.'},
  'subtitle': 'Latest Financial News for dis',
  'subtitle_detail': {'type': 'text/html',
   'language': None,
   'base': '',
   'value': 'Latest Financial News for dis'},
  'image': {'height': 45,
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'http://finance.yahoo.com/q/h?s=dis'}],
   'link': 'http://finance.yahoo.com/q/h?s=dis',
   'title': 'Yahoo! Finance: dis News',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': '',
    'value': 'Yahoo! Finance: dis News'},
   'href': 'http://l.yimg.com/a/i/brand/purplelogo/uh/us/fin.gif',
   'width': 144},
  'language': 'en-US',
  'updated': 'Sat, 04 Apr 2020 05:45:06 +0000',
  'updated_parsed': time.struct_time(tm_year=2020, tm_mon=4, tm_mday=4, tm_hour=5, tm_min=45, tm_sec=

In [ ]:
import asyncio
from contextlib import closing
import time

import aiohttp


async def fetch_page(session, host, port=8000, wait=0):
    url = '{}:{}/{}'.format(host, port, wait)
    with aiohttp.Timeout(10):
        async with session.get(url) as response:
            assert response.status == 200
            return await response.text()


def get_multiple_pages(host, waits, port=8000, show_time=True):
    tasks = []
    pages = []
    start = time.perf_counter()
    with closing(asyncio.get_event_loop()) as loop:
        with aiohttp.ClientSession(loop=loop) as session:
            for wait in waits:
                tasks.append(fetch_page(session, host, port, wait))
            pages = loop.run_until_complete(asyncio.gather(*tasks))
    duration = time.perf_counter() - start
    sum_waits = sum(waits)
    if show_time:
        msg = 'It took {:4.2f} seconds for a total waiting time of {:4.2f}.'
        print(msg.format(duration, sum_waits))
    return pages

pages = get_multiple_pages(host='http://localhost', port='8000', waits=[1, 5, 3, 2])
for page in pages:
    print(page)


In [68]:
import feedparser

d = feedparser.parse('http://finance.yahoo.com/rss/headline?s=yhoo')
d
# d['entries'][0]
# print(d.modified)
# d2 = feedparser.parse('https://finance.yahoo.com/rss/industry?s=yhoo', etag=d.etag)

{'feed': {'rights': 'Copyright (c) 2020 Yahoo! Inc. All rights reserved.',
  'rights_detail': {'type': 'text/plain',
   'language': None,
   'base': 'https://feeds.finance.yahoo.com/rss/2.0/headline?s=yhoo&region=US&lang=en-US',
   'value': 'Copyright (c) 2020 Yahoo! Inc. All rights reserved.'},
  'subtitle': 'Latest Financial News for yhoo',
  'subtitle_detail': {'type': 'text/html',
   'language': None,
   'base': 'https://feeds.finance.yahoo.com/rss/2.0/headline?s=yhoo&region=US&lang=en-US',
   'value': 'Latest Financial News for yhoo'},
  'image': {'height': 45,
   'links': [{'rel': 'alternate',
     'type': 'text/html',
     'href': 'http://finance.yahoo.com/q/h?s=yhoo'}],
   'link': 'http://finance.yahoo.com/q/h?s=yhoo',
   'title': 'Yahoo! Finance: yhoo News',
   'title_detail': {'type': 'text/plain',
    'language': None,
    'base': 'https://feeds.finance.yahoo.com/rss/2.0/headline?s=yhoo&region=US&lang=en-US',
    'value': 'Yahoo! Finance: yhoo News'},
   'href': 'http://l.yi

In [70]:
d['entries'][0]['published_parsed']

time.struct_time(tm_year=2019, tm_mon=4, tm_mday=9, tm_hour=18, tm_min=29, tm_sec=5, tm_wday=1, tm_yday=99, tm_isdst=0)

In [77]:
import time
x = [time.strptime("30 Nov 10", "%d %b %y"), time.strptime("30 Nov 00", "%d %b %y")]
# x.sort()

[time.struct_time(tm_year=2010, tm_mon=11, tm_mday=30, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=334, tm_isdst=-1),
 time.struct_time(tm_year=2000, tm_mon=11, tm_mday=30, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=335, tm_isdst=-1)]

In [20]:
# from newspaper import Article

# url = 'https://finance.yahoo.com/news/u-deaths-top-1-000-034816333.html'
# article = Article(url)
# article.download()
# article.parse()
# print(article.authors)
# print(article.publish_date)
# print(article.text)
# print(article.nlp())

In [15]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://es:9200")
es.info()

{'name': '66dfbd372aa5',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'i_0M-3txTu6W9VNq8rpZXw',
 'version': {'number': '7.6.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'aa751e09be0a5072e8570670309b1f12348f023b',
  'build_date': '2020-02-29T00:15:25.529771Z',
  'build_snapshot': False,
  'lucene_version': '8.4.0',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [28]:
import twint

c = twint.Config()

# c.Username = "YahooFinance"
# c.Username = "CNBC"
c.Search = '$AAPL'
c.Limit = 10
# c.Since= '2017-12-27'
# c.Until = '2017-12-31'
# c.Store_csv = True
# c.Output = "none"
c.Elasticsearch = "http://es:9200"
c.Debug = True

twint.run.Search(c)

[+] Indexing to Elasticsearch @ http://es:9200
....................

In [12]:
c